In [98]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
# import psycopg2

# from config import db_password

import time

In [99]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [103]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')


    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}wikipedia_movies.json', mode='r') as file:
        wiki_movies_file = json.load(file)
        
    movies_json = json.load(wiki_movies_file)
    # 3. Write a list comprehension to filter out TV shows.
    movies_only = [movie for movie in movies_json if('Director' in movie or 'Directed by' in movie) 
                    and 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    wiki_movies_clean = [clean_movie(movie) for movie in movies_only]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_clean)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset = 'imdb_id', inplace = True)
    except Exception as e: print(e)
    
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_no_null = [column for column in wiki_movies_df.columns 
                   if wiki_movies_df[column].isnull().sum() < (0.9 * len(wiki_movies_df))]
    wiki_movies_df = wiki_movies_df[wiki_no_null]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if x == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"


    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        if type(s) != str:
                return np.nan

        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

             s = re.sub('\$|\s|[a-zA-Z]','', s)

             value = float(s) * 10**6

             return value

        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            s = re.sub('\$|\s|[a-zA-Z]','', s)

            value = float(s) * 10**9

            return value

        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            s = re.sub('\$|,','', s)

            value = float(s)

            return value

        else:
            return np.nan
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
        wiki_movies_df['Box_office'] = box_office.str.extract(f'({form_one}|{form_two})', \
                                                          flags=re.IGNORECASE)[0].apply(parse_dollars)
    
        wiki_movies_df.drop('Box office', axis=1, inplace=True)
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
        budget = wiki_movies_df['Budget'].dropna().apply(lambda x: ' '.join(x) if x == list else x)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.

        release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
        running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
          
        
    # Return three variables. The first is the wiki_movies_df DataFrame

    return wiki_movies_df, kaggle_metadata, ratings 


SyntaxError: invalid syntax (4222246499.py, line 14)

In [101]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = "D://DataClass/Challenge 8/"
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [102]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

AttributeError: 'list' object has no attribute 'read'

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [ ]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [ ]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()